In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore')
%matplotlib inline
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
kc = pd.read_csv('./source/kc_house_data.csv', parse_dates=[1,14])

In [42]:
## Cleaning the data

# waterfront, although incomplete, hasn't been altered due to the complexity of the variable assignment

# sqft_basement: 454 '?' values changed to zero.
properties_data_altered = {}
for i in range(0,len(kc['id'])-1):
        if kc['sqft_basement'][i] == '?':
            kc['sqft_basement'][i] = 0
        properties_data_altered[kc['id'][i]] = 'sqft_basement'

# 33 bedrooms house? It is a typo (the 11 bedrooms one checks out though)
properties_data_altered[kc[kc['bedrooms']==33].iloc[0]['id']] = '33_bedrooms'
kc['bedrooms'] = kc['bedrooms'].replace(33,3)

# view, 63 empties changed to zeros
for i in kc[kc['view'].isnull()]['id']:
    properties_data_altered[i] = 'views'
kc['view'] = kc['view'].fillna(0)

# yr_renovated, 16900 zeros changed to NaN
for i in kc[kc['yr_renovated']=='0.0']['id']:
    properties_data_altered[i] = 'yr_renovated'
kc['yr_renovated'] = kc['yr_renovated'].replace('0.0', np.nan)

# sorting out data types:
kc['id'] = kc['id'].astype(str)
kc['bathrooms'] = kc['bathrooms'].astype(str)
kc['bedrooms'] = kc['bedrooms'].astype(str)
kc['condition'] = kc['condition'].astype(str)
kc['floors'] = kc['floors'].astype(str)
kc['grade'] = kc['grade'].astype(str)
kc['sqft_basement'] = kc['sqft_basement'].astype(float)
kc['view'] = kc['view'].astype(str)
kc['waterfront'] = kc['waterfront'].astype(str)
kc['yr_renovated'] = pd.to_datetime(kc['yr_renovated'])
kc['zipcode'] = kc['zipcode'].astype(str)

# split dataframes: properties sold more than once in the timeframe vs all properties without duplicates
kc_duplicated = kc[kc.duplicated(['id'],keep=False)]
kc = kc.sort_values('date').drop_duplicates('id',keep='last')

<ipython-input-42-050c3f8d7ae5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kc['sqft_basement'][i] = 0
/home/antoine/.local/lib/python3.8/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 7316 to 16580
Data columns (total 21 columns):
id               21420 non-null object
date             21420 non-null datetime64[ns]
price            21420 non-null float64
bedrooms         21420 non-null object
bathrooms        21420 non-null object
sqft_living      21420 non-null int64
sqft_lot         21420 non-null int64
floors           21420 non-null object
waterfront       21420 non-null object
view             21420 non-null object
condition        21420 non-null object
grade            21420 non-null object
sqft_above       21420 non-null int64
sqft_basement    21420 non-null float64
yr_built         21420 non-null datetime64[ns]
yr_renovated     17607 non-null datetime64[ns]
zipcode          21420 non-null object
lat              21420 non-null float64
long             21420 non-null float64
sqft_living15    21420 non-null int64
sqft_lot15       21420 non-null int64
dtypes: datetime64[ns](3), float64(4), int64(5

## Things to look at:
* split per zipcodes
* check the price growth over time to figure out the up and coming areas
* ID zipcodes with waterfront and see how it affects average price
* Where can we make money?/What is the most underrated property type? 
* Plot the schools and trasnports (although public transport in the US might not be 